# Study Kasus Naive Bayes (Exams)

Kelompok x:
- 10120262 Shania Nur Lutfia 
- 10120263 Elsa Nur Rahmadiani
- 10120706 Hilmi Muhammad
- 10120708 Taufiq Qurohman Ruki

# Import Library

In [1]:
import pandas as pd

# Data Loading

Tahap ini dilakukan untuk melihat isi dataset yang digunakan, yaitu dataset tentang exams

In [2]:
df = pd.read_excel('exams(NAIVE BAYES).xlsx')
df.head()

,gender,race/ethnicity,parental level of education,test preparation course,math score,math score.1,reading score,reading score.1,writing score,writing score.1,Class_lunch
0,male,group A,high school,completed,C,67,C,67,C,63,standard
1,female,group D,some high school,none,E,40,D,59,D,55,free/reduced
2,male,group E,some college,none,D,59,D,60,D,50,free/reduced
3,male,group B,high school,none,B,77,B,78,C,68,standard
4,male,group E,associate's degree,completed,B,78,B,73,C,68,standard


In [3]:
baris, kolom = df.shape
print("Jumlah Baris: ", baris)
print("Jumlah Kolom: ", kolom)

Jumlah Baris:  1000
Jumlah Kolom:  11


Dataset exams terdiri dari 1000 baris dan 11 kolom

In [4]:
df.columns

Index(['gender', 'race/ethnicity', 'parental level of education',
       'test preparation course', 'math score', 'math score.1',
       'reading score', 'reading score.1', 'writing score', 'writing score.1',
       'Class_lunch'],
      dtype='object')

In [5]:
# cek tipe data
df.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
test preparation course        object
math score                     object
math score.1                    int64
reading score                  object
reading score.1                 int64
writing score                  object
writing score.1                 int64
Class_lunch                    object
dtype: object

# Preprocessing

Tahap ini dilakukan untuk memilih fitur mana saja yang akan dipakai. Selain itu, dapat dilakukan juga transformasi data maupun pembersihan data jika diperlukan.

In [6]:
# hapus kolom yang tidak perlu
df.drop(['gender','math score.1','reading score.1','writing score.1'], axis=1, inplace=True)

In [7]:
df.sample(7)

,race/ethnicity,parental level of education,test preparation course,math score,reading score,writing score,Class_lunch
349,group B,some high school,completed,B,A,A,standard
144,group B,master's degree,completed,A,B,C,standard
955,group D,associate's degree,completed,A,B,A,standard
137,group B,associate's degree,completed,E,C,C,standard
36,group C,some college,completed,D,B,B,standard
8,group D,high school,none,C,D,C,standard
403,group B,some college,completed,C,C,C,standard


Setelah dilakukan pemilihan fitur, selanjutnya akan dilakukan pembagian dataset untuk train dan testing

In [8]:
# data dibagi menjadi data training dan data testing, dengan perbandingan 80:20
train = df.sample(frac=0.8, random_state=200)
test = df.drop(train.index)

# menampilkan data training
print(train.shape)

# menampilkan data testing
print(test.shape)

(800, 7)
(200, 7)


Dikarenakan data test akan diuji coba, maka isi kolom 'Class_lunch' akan dikosongkan

In [9]:
# hapus isi kolom 'Class_lunch' 
test['Class_lunch'] = test['Class_lunch'].replace(['standard','free/reduced'], ['',''])

In [10]:
test.sample(7)

,race/ethnicity,parental level of education,test preparation course,math score,reading score,writing score,Class_lunch
913,group C,some college,completed,D,C,C,
774,group A,some college,none,B,C,C,
611,group A,high school,completed,C,A,B,
991,group C,associate's degree,none,A,A,A,
962,group B,some high school,completed,E,E,E,
641,group C,high school,none,B,B,C,
698,group C,some college,completed,B,A,B,


Tentukan nilai prior:
- n(T) = jumlah semua data
- n(A1) = jumlah data Class_lunch = standar
- n(A2) = jumlah data Class_lunch = free/reduced
- n(PA1) = nilai A1/T
- n(PA2) = nilai A2/T

# Hitung Nilai Prior

In [11]:
# Target Class
target_class = 'Class_lunch'

# Tampilkan Value dari target class
target_class_values = train[target_class].unique()
print(f"Nilai unik dari target class: {target_class_values}\n")

# Menentukan Nilai Prior Probability
prior = {}

# Jumlah total data
n_T = len(train)
prior['n(T)'] = n_T
print(f"Jumlah total data (n(T)): {n_T}")

# Jumlah data untuk setiap nilai target class
for idx, target in enumerate(target_class_values):
    n_Ai = len(train[train[target_class] == target])
    prior[f'n(A{idx + 1})'] = n_Ai
    print(f"Jumlah data dengan target class {target} (n(A{idx + 1})): {n_Ai}")

# Menghitung prior probability
for idx, target in enumerate(target_class_values):
    n_Ai = len(train[train[target_class] == target])
    p_Ai = n_Ai / n_T
    prior[f'P(A{idx + 1})'] = p_Ai
    print(f"Prior probability untuk target class {target} (P(A{idx + 1})): {p_Ai}")


Nilai unik dari target class: ['standard' 'free/reduced']

Jumlah total data (n(T)): 800
Jumlah data dengan target class standard (n(A1)): 521
Jumlah data dengan target class free/reduced (n(A2)): 279
Prior probability untuk target class standard (P(A1)): 0.65125
Prior probability untuk target class free/reduced (P(A2)): 0.34875


# Tabel Kontigensi


In [12]:
# Kontigensi dari setiap atribut
attributes = train.columns.tolist()

for attribute in attributes:
    if attribute != target_class:
        print("-" * 60)
        print(f"Kontigensi dari atribut \"{attribute}\"")
        print("-" * 60)
        for target in target_class_values:
            print(f"{target_class} \"{target}\"")
            attribute_class_value = train[attribute].unique().tolist()
            total = 0
            print(f"{attribute} :")
            for attribute_value in attribute_class_value:
                n_Ai = len(train[(train[attribute] == attribute_value) & (train[target_class] == target)])
                print(f"{attribute_value} = {n_Ai}")
                total += n_Ai
            print(f"Total = {total}\n")

------------------------------------------------------------
Kontigensi dari atribut "race/ethnicity"
------------------------------------------------------------
Class_lunch "standard"
race/ethnicity :
group B = 109
group C = 166
group E = 53
group D = 152
group A = 41
Total = 521

Class_lunch "free/reduced"
race/ethnicity :
group B = 54
group C = 94
group E = 45
group D = 65
group A = 21
Total = 279

------------------------------------------------------------
Kontigensi dari atribut "parental level of education"
------------------------------------------------------------
Class_lunch "standard"
parental level of education :
master's degree = 37
some college = 107
associate's degree = 99
some high school = 98
high school = 108
bachelor's degree = 72
Total = 521

Class_lunch "free/reduced"
parental level of education :
master's degree = 23
some college = 73
associate's degree = 56
some high school = 50
high school = 52
bachelor's degree = 25
Total = 279

------------------------------

# Pelatihan P(B|Ai)

Pelatihan model melibatkan perhitungan probabilitas kondisional P(B|Ai), di mana B adalah kelas target yang akan diprediksi, dan Ai adalah fitur atau atribut yang akan digunakan sebagai dasar prediksi.

**P(B∣Ai)= P(Ai)/ P(Ai∣B)⋅P(B)**
​


In [13]:
# Hasil Pelatihan P(B|Ai)
print("Hasil Pelatihan P(B|Ai)\n")

for attribute in attributes:
    if attribute != target_class:
        print("-" * 60)
        print(f"Hasil Pelatihan P(B|Ai) dari atribut \"{attribute}\"")
        print("-" * 60)
        for baris, target in enumerate(target_class_values):
            print(f"{attribute}")
            print(f"{target_class} \"{target}\" => P(B|A{baris + 1})")
            attribute_class_value = train[attribute].unique().tolist()
            total = 0
            print(f"{attribute} :")
            for attribute_value in attribute_class_value:
                n_Ai = len(train[(train[attribute] == attribute_value) & (train[target_class] == target)])
                n_Ai_target = len(train[train[target_class] == target])
                p_B_Ai = n_Ai / n_Ai_target
                print(f"{attribute_value} = {p_B_Ai}")
                total += p_B_Ai
            print(f"Hasil : {total}")
            if(baris < len(target_class_values) - 1):
                print("-" * 60)
        print("\n")


hasil_pelatihan = {}

for target in target_class_values:
    hasil_pelatihan[target] = {}
    for attribute in attributes:
        if attribute != target_class:
            hasil_pelatihan[target][attribute] = {}
            attribute_class_value = train[attribute].unique().tolist()
            for attribute_value in attribute_class_value:
                n_Ai = len(train[(train[attribute] == attribute_value) & (train[target_class] == target)])
                n_Ai_target = len(train[train[target_class] == target])
                p_B_Ai = n_Ai / n_Ai_target
                hasil_pelatihan[target][attribute][attribute_value] = p_B_Ai

print(hasil_pelatihan)

Hasil Pelatihan P(B|Ai)

------------------------------------------------------------
Hasil Pelatihan P(B|Ai) dari atribut "race/ethnicity"
------------------------------------------------------------
race/ethnicity
Class_lunch "standard" => P(B|A1)
race/ethnicity :
group B = 0.2092130518234165
group C = 0.31861804222648754
group E = 0.1017274472168906
group D = 0.29174664107485604
group A = 0.07869481765834933
Hasil : 1.0
------------------------------------------------------------
race/ethnicity
Class_lunch "free/reduced" => P(B|A2)
race/ethnicity :
group B = 0.1935483870967742
group C = 0.33691756272401435
group E = 0.16129032258064516
group D = 0.23297491039426524
group A = 0.07526881720430108
Hasil : 1.0


------------------------------------------------------------
Hasil Pelatihan P(B|Ai) dari atribut "parental level of education"
------------------------------------------------------------
parental level of education
Class_lunch "standard" => P(B|A1)
parental level of education 

# Testing

Cek data testing yang telah disiapkan sebelumnya

In [14]:
# cek data testing
test.head()

,race/ethnicity,parental level of education,test preparation course,math score,reading score,writing score,Class_lunch
0,group A,high school,completed,C,C,C,
2,group E,some college,none,D,D,D,
6,group A,bachelor's degree,none,C,D,C,
7,group E,some college,completed,A,A,A,
10,group E,some college,completed,A,A,A,


In [15]:
# test model dengan data testing
for idx, row in test.iterrows():
    print("=" * 50)
    print(f"Data testing ke-{idx + 1}")
    print("=" * 50)

    max_probability = 0
    predicted_class = None

    for target in target_class_values:
        total = 1
        for attribute in attributes:
            if attribute != target_class:
                # print(f"{attribute} = {row[attribute]}")
                total *= hasil_pelatihan[target][attribute][row[attribute]]

        print(f"Hasil : {total}")
        print(f"{target_class}: \"{target}\"")
        # print("=" * 50)

        # Membandingkan probabilitas kelas untuk memprediksi kelas terbaik
        if total > max_probability:
            max_probability = total
            predicted_class = target

    print(f"\nPrediksi kelas: {predicted_class}\n")


Data testing ke-1
Hasil : 7.327620229800603e-05
Class_lunch: "standard"
Hasil : 5.695642140105048e-05
Class_lunch: "free/reduced"

Prediksi kelas: standard

Data testing ke-3
Hasil : 5.680524172555807e-05
Class_lunch: "standard"
Hasil : 0.00038813404738205487
Class_lunch: "free/reduced"

Prediksi kelas: free/reduced

Data testing ke-7
Hasil : 7.372388749377928e-05
Class_lunch: "standard"
Hasil : 5.065334039083278e-05
Class_lunch: "free/reduced"

Prediksi kelas: standard

Data testing ke-8
Hasil : 0.00014480746715148739
Class_lunch: "standard"
Hasil : 2.055712415907561e-05
Class_lunch: "free/reduced"

Prediksi kelas: standard

Data testing ke-11
Hasil : 0.00014480746715148739
Class_lunch: "standard"
Hasil : 2.055712415907561e-05
Class_lunch: "free/reduced"

Prediksi kelas: standard

Data testing ke-18
Hasil : 0.00017397184435476198
Class_lunch: "standard"
Hasil : 0.00032603900199447487
Class_lunch: "free/reduced"

Prediksi kelas: free/reduced

Data testing ke-30
Hasil : 1.66541298137499